In [ ]:
import xml.etree.ElementTree as ET
from transformers import BertTokenizer, BertConfig
from transformers import BertModel
import nltk
import pandas as pd
import numpy as np
import re
import nltk
from sklearn import model_selection 
from sklearn.linear_model import LogisticRegression
import pickle
import string
from transformers import BertTokenizer, BertConfig
from transformers import BertModel
import torch
from nltk.corpus import stopwords
import os

In [ ]:
def get_token_positions(input_text,token_list,start_from_pos=0):
    pos_list = []                    
    name_to_match = input_text.lower().replace(" ","")
    remaining_name = input_text.lower().replace(" ","")
    name = ""
    count = start_from_pos
    for entry in token_list[start_from_pos:]:
        if(remaining_name.startswith(entry.strip("##").lower())):
            pos_list.append(count)
            remaining_name = remaining_name[len(entry.strip("##").lower()):]
            name = name + entry.strip("##").lower()
            if(name == name_to_match):
                break
        else:
            pos_list = []
            name = ""
            remaining_name = name_to_match
            if(remaining_name.startswith(entry.strip("##").lower())):                                 
                pos_list.append(count)                                                                
                remaining_name = remaining_name[len(entry.strip("##").lower()):]
                name = name + entry.strip("##").lower()    
                if(name == name_to_match):                                   
                    break

        count = count + 1

    return pos_list

In [ ]:
def process_string(string_input, padding_length):
    #string_input = re.sub(r"(\w)(["+ string.punctuation +"])", r"\1 \2 ", string_input)
    #string_input = string_input.replace("."," . ")

    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = tokenizer.tokenize(string_input)
    
    config = BertConfig.from_pretrained('C:/Users/itsma/Documents/BERT_models/NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12')
    config.output_hidden_states = True

    bert_tokenizer = BertTokenizer.from_pretrained('C:/Users/itsma/Documents/BERT_models/NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12')
    bert_model = BertModel.from_pretrained("C:/Users/itsma/Documents/BERT_models/NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12",config=config)
    #bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    #bert_model = BertModel.from_pretrained('bert-base-uncased')
    #stop_words = set(stopwords.words('english')) 
    
    positions_covered = 0
    word_list = []
    
    for index in range(len(sentences)):
        new_dict_sentence = {}
        sentence = sentences[index]
        new_dict_sentence["sentence"] = sentence
        new_dict_sentence["padding_length"] = padding_length
        start_index_bert = max(0,index-padding_length)
        end_index_bert = min(len(sentences),index+padding_length)

        bert_input = ' '.join(sentences[start_index_bert:(end_index_bert+1)])

        input_ids = torch.tensor(bert_tokenizer.encode(bert_input)).unsqueeze(0)  
        outputs = bert_model(input_ids)
        bert_vector = outputs[2][0][0].data.numpy()
        bert_tokens = bert_tokenizer.tokenize(bert_input)

        '''
        bert_vec_list = []
        for entry in range(0,len(outputs[0][0])):
            bert_vec_list.append(outputs[0][0][entry].data.numpy())

        bert_vector_sentence = np.mean(bert_vec_list,axis=0)
        '''
        start_pos = 0
        prior_pos = get_token_positions(' '.join(sentences[start_index_bert:index]),bert_tokens)
        
        if(len(prior_pos)>0):
            start_pos = max(prior_pos)
            
        tokens = nltk.word_tokenize(sentence)
        pos_tokens = nltk.pos_tag(tokens)

        sentence_covered = ''
        
        for token in pos_tokens:
            new_dict = {}
            current_word = token[0]
            new_dict["word"] = current_word
            token_position = string_input.find(current_word, positions_covered)
            spaces_between = string_input[positions_covered:token_position] 
            sentence_covered = sentence_covered + spaces_between + current_word
            positions_covered = token_position + len(current_word)
            new_dict["begin_pos"] = token_position
            new_dict["end_pos"] = positions_covered

            bert_token_positions = get_token_positions(current_word,bert_tokens,start_pos)
            vec_list = []
            if(len(bert_token_positions)==0):
                continue
            start_pos = bert_token_positions[-1] + 1

            for entry in bert_token_positions:
                vec_list.append(bert_vector[entry])
            
            bert_vector_word = np.mean(vec_list,axis=0)
            new_dict["keyword_vector"] = bert_vector_word

            word_list.append(new_dict)
    
    return word_list

In [ ]:
def is_available_in_pos_list(dict_list, begin_pos, end_pos):
    for entry in dict_list:
        if(entry["start"]<=begin_pos and entry["end"]>=end_pos):
            return True
    
    return False


In [ ]:
class Discharge_note():
    def __init__(self, root):
        self.xml_root = root
        
    def process_note(self):
        root = self.xml_root
        text_section = root.find('TEXT')
        text = text_section.text
        
        self.processed_text = process_string(text,1)

        tag_section = root.find('TAGS')
        event_list = []
        timex_list = []
        tlink_list = []
        sectime_list = []
        for child in tag_section:
            if(child.tag=='EVENT'):
                event_list.append(child.attrib)
            elif(child.tag=='TIMEX3'):
                timex_list.append(child.attrib)
            elif(child.tag=='TLINK'):
                tlink_list.append(child.attrib)
            elif(child.tag=='SECTIME'):
                sectime_list.append(child.attrib)

        for sub in event_list:
            sub["start"] = int(sub["start"])
            sub["end"] = int(sub["end"])

        for sub in timex_list:
            sub["start"] = int(sub["start"])
            sub["end"] = int(sub["end"])

        for sub in sectime_list:
            sub["start"] = int(sub["start"])
            sub["end"] = int(sub["end"])


        event_list = sorted(event_list, key = lambda i: i['start'])
        timex_list = sorted(timex_list, key = lambda i: i['start'])
        sectime_list = sorted(sectime_list, key = lambda i: i['start'])
    
        for entry in self.processed_text:
            event_entry_available = is_available_in_pos_list(event_list,entry["begin_pos"],entry["end_pos"])
            timex_entry_available = is_available_in_pos_list(timex_list,entry["begin_pos"],entry["end_pos"])
            sectime_entry_available = is_available_in_pos_list(sectime_list,entry["begin_pos"],entry["end_pos"])

            entry.update({"event_flag":1 if event_entry_available else 0})
            entry.update({"timex_flag":1 if timex_entry_available else 0})
            entry.update({"sectime_flag":1 if sectime_entry_available else 0})
        

In [ ]:
word_list = []
count = 0
for file in os.listdir("C:/Users/itsma/Documents/Capstone project/2012-07-15.original-annotation.release/"):
    if file.endswith(".xml"):
        try:
            file_name = os.path.join("C:/Users/itsma/Documents/Capstone project/2012-07-15.original-annotation.release/", file)
            tree = ET.parse(file_name)
            root = tree.getroot()
            discharge_note = Discharge_note(root)
            discharge_note.process_note()
            word_list.extend(discharge_note.processed_text)
            count = count + 1
            print(count)
            
        except:
            count = count + 1
            print("Error in processing file:",file)

In [ ]:
len(word_list)

In [ ]:
word_list_df = pd.DataFrame(word_list)

In [ ]:
pos_data = word_list_df[word_list_df["sectime_flag"]==1]
neg_data = word_list_df[word_list_df["sectime_flag"]==0]

In [ ]:
pos_data_volume = len(pos_data)
neg_data = neg_data.sample(pos_data_volume)

In [ ]:
len(pos_data)

In [ ]:
combined_data = pd.concat([pos_data,neg_data])
X = np.vstack(list(combined_data["keyword_vector"]))                                
y = combined_data.sectime_flag  
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.10, stratify=y, random_state=0)

clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='ovr').fit(X_train, y_train)

print("Score on test data:",clf.score(X_test,y_test))

In [ ]:
from sklearn import metrics

In [ ]:
predictions = clf.predict(X_test)

cm = metrics.confusion_matrix(y_test, predictions)
print(cm)